## Q1. You are working on a machine learning project where you have a dataset containing numerical and categorical features. You have identified that some of the features are highly correlated and there are missing values in some of the columns. You want to build a pipeline that automates the feature engineering process and handles the missing values.
#### Design a pipeline that includes the following steps
 - Use an automated feature selection method to identify the important features in the dataset
 - Create a numerical pipeline that includes the following steps
 - Impute the missing values in the numerical columns using the mean of the column values
 - Scale the numerical columns using standardisation
 - Create a categorical pipeline that includes the following steps
 - Impute the missing values in the categorical columns using the most frequent value of the column
 - One-hot encode the categorical columns
 - Combine the numerical and categorical pipelines using a ColumnTransformer
 - Use a Random Forest Classifier to build the final model
 - Evaluate the accuracy of the model on the test dataset.
## Note: Your solution should include code snippets for each step of the pipeline, and a brief explanation of each step. You should also provide an interpretation of the results and suggest possible improvements for the pipeline.

In [1]:
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
import seaborn as sns

In [2]:
df = sns.load_dataset("diamonds")

In [3]:
df = df.head(10000)

In [4]:
df.columns

Index(['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'price', 'x', 'y',
       'z'],
      dtype='object')

In [5]:
df.isnull().sum()

carat      0
cut        0
color      0
clarity    0
depth      0
table      0
price      0
x          0
y          0
z          0
dtype: int64

In [6]:
X = df.drop("price", axis=1)
y = df["price"]

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [8]:
imp_features = SelectKBest(score_func=f_classif, k=5)

In [9]:
categorical_features = ["cut", "color", "clarity"]
numerical_features = ["carat", "depth", "table", "x", "y", "z"]

In [10]:
numerical_pipeline = Pipeline(
    steps=[
        ("scaler" , StandardScaler())
    ]
)

categorical_pipeline = Pipeline(
    steps=[
        ("encoder" , OneHotEncoder())
    ]
)

In [11]:
preprocessor = ColumnTransformer(
    [
        ("numerical", numerical_pipeline, numerical_features),
        ("categorical", categorical_pipeline, categorical_features)
    ]
)

In [12]:
final_pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("feature_selection", imp_features),
    ("Random_forest_classifier", RandomForestClassifier(random_state=42))
])

In [13]:
final_pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numerical',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['carat', 'depth', 'table',
                                                   'x', 'y', 'z']),
                                                 ('categorical',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder())]),
                                                  ['cut', 'color',
                                                   'clarity'])])),
                ('feature_selection', SelectKBest(k=5)),
                ('Random_forest_classifier',
                 RandomForestClassifier(random_state=42))])

In [ ]:
final_pipeline.fit(X_train, y_train)

In [ ]:
y_pred = final_pipeline.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(y_pred, y_test)

## 

##  Q2. Build a pipeline that includes a random forest classifier and a logistic regression classifier, and then use a voting classifier to combine their predictions. Train the pipeline on the iris dataset and evaluate its accuracy.
## Answer 

In [200]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import seaborn as sns

In [201]:
df = sns.load_dataset("iris")

In [202]:
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [203]:
df["species"].nunique()

3

In [204]:
encoder = LabelEncoder()

In [205]:
encoded = encoder.fit(df["species"])

In [206]:
df["species"] = encoded.transform(df["species"])

In [207]:
df

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [208]:
X = df.iloc[:, :-1]
y = df["species"]

In [209]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [210]:
from sklearn.ensemble import VotingClassifier

In [211]:
rnf = RandomForestClassifier(random_state=42)
lgr = LogisticRegression(random_state=42)

In [212]:
vc = VotingClassifier(estimators=[
    ("random_forest",rnf), ("logistic_regression", lgr)],
    voting="hard")

In [213]:
vc.fit(X_train, y_train)

VotingClassifier(estimators=[('random_forest',
                              RandomForestClassifier(random_state=42)),
                             ('logistic_regression',
                              LogisticRegression(random_state=42))])

In [214]:
y_pred = vc.predict(X_test)

In [216]:
accuracy_score(y_pred, y_test)

1.0